In [13]:
import pgserver
db = pgserver.get_server(r'C:\Users\paul_ip\Documents\GitHub\hk-address-translate\pgdata')

In [14]:
print(db.psql("SELECT 1+1 as res;"))

 res 
-----
   2
(1 row)




In [ ]:
#terminate the server connection
db.cleanup()